In [1]:
# install kaggle
# pip install kaggle

     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     -------------- ----------------------- 30.7/82.7 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 82.7/82.7 kB 928.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
   --------------------------------------- 162.8/162.8 kB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
   ---------------------------------------- 78.2/78.2 kB ? eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105796 sha256=e2b90435c3c28e9f7fd4b16339b4cc26aa4a9e832732bd7a40dfa3e64234c243
  Stored in directory: c:\users\ankit\appdata\local\pip\cache\wheels\9f\af\22\bf406f913dc7506a485e60dce8143741abd0a92a19337d83a3
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [2]:
# kaggle dataset download into url into .zip
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:01<00:00, 204kB/s]
100%|██████████| 200k/200k [00:01<00:00, 204kB/s]


In [3]:
# extract csv from .zip file
import zipfile
zip_ref = zipfile.ZipFile("orders.csv.zip")
zip_ref.extractall() #extract file to dir
zip_ref.close()

In [5]:
# read data from the file and handle null values
import pandas as pd
df =  pd.read_csv('orders.csv',na_values = ['Not Available','unknown'])
# df.info()
# df.head(10)
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [7]:
# # rename columns names ..make them lower case and replace space with underscores
# # df.rename(columns={'Order Id':'order_id','City':'city'})
# df.columns = df.columns.str.lower()
# df.columns = df.columns.str.replace(' ','_')
# df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [8]:
# derive new columns discount, sale price and profit
# df['discount']=df['list_price']*df['discount_percent']*.01
# df['sale_price'] = df['list_price']-df['discount']
df['profit'] = df['sale_price'] - df['cost_price']

In [9]:
# convert order date from object data to datetime
# df.dtypes
# df['order_date']=pd.to_datetime(df['order_date'],format='%Y-%m-%d')
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [10]:
# drop cost price, list price and discount percent columns
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

In [24]:
# load the data into sql server using replace option
# using vscode extension sqlite I have created a database called da.db
# pip install sqlalchemy
import sqlalchemy as sal
from sqlalchemy import text
# Create an SQLite engine
engine = sal.create_engine('sqlite:///da.db')
conn = engine.connect()
# Load the data into the SQLite database using the 'replace' option
# df.to_sql('df_orders',con=conn,index=False,if_exists='replace')
df.to_sql('df_orders',con=conn,index=False,if_exists='append')

## Query to find the top 10 highest revenue-generating products
# top_10_query = '''
# SELECT product_id, SUM(sale_price) AS sales
# FROM df_orders
# GROUP BY product_id
# ORDER BY sales DESC
# LIMIT 10;
# '''

# Execute the query and print the results
# with engine.connect() as conn:
#     result = conn.execute(text(top_10_query))
#     for row in result:
#         print(row)


('TEC-CO-10004722', 59514.0)
('OFF-BI-10003527', 26525.300000000003)
('TEC-MA-10002412', 21734.4)
('FUR-CH-10002024', 21096.2)
('OFF-BI-10001359', 19090.2)
('OFF-BI-10000545', 18249.0)
('TEC-CO-10001449', 18151.2)
('TEC-MA-10001127', 17906.4)
('OFF-BI-10004995', 17354.8)
('OFF-SU-10000151', 16325.8)


In [60]:
# del df['discount']
# by default 5 rows will show
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
